#  Hệ Thống Gợi Ý Ngành Học - Huấn Luyện Mô Hình
## Training với dữ liệu hiện có của bạn

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Đã import tất cả thư viện cần thiết")

## Bước 1: Kiểm tra tệp dữ liệu của bạn

In [ ]:
def check_data_files():
    """Kiểm tra xem các tệp dữ liệu có tồn tại không"""
    print("🔍 KIỂM TRA TỆP DỮ LIỆU...")
    
    # Kiểm tra các file trong thư mục data
    data_files = []
    if os.path.exists('data'):
        print("📁 KIỂM TRA THƯ MỤC 'data':")
        for file in os.listdir('data'):
            file_path = os.path.join('data', file)
            file_size = os.path.getsize(file_path) / 1024
            print(f"   ✅ {file}: {file_size:.1f} KB")
            data_files.append(file_path)
    else:
        print("❌ Thư mục 'data' không tồn tại")
    
    # Kiểm tra các file trong thư mục gốc
    print("\n📁 KIỂM TRA THƯ MỤC GỐC:")
    root_files = [
        'final_majors_data.csv',
        'tfidf_vectorizer.pkl',
        'notebook.ipynb'
    ]
    
    for file in root_files:
        if os.path.exists(file):
            file_size = os.path.getsize(file) / 1024
            print(f"   ✅ {file}: {file_size:.1f} KB")
        else:
            print(f"   ❌ {file}: KHÔNG TÌM THẤY")
    
    return data_files

# Kiểm tra tệp
available_files = check_data_files()

## Bước 2: Kiểm tra mô hình đã huấn luyện

In [ ]:
def check_existing_model():
    """Kiểm tra xem đã có mô hình huấn luyện chưa"""
    print("\n KIỂM TRA MÔ HÌNH ĐÃ HUẤN LUYỆN (VIETNAMESE)...")
    
    if os.path.exists('tfidf_vectorizer_vi.pkl'):
        print("✅ Tìm thấy mô hình đã huấn luyện: tfidf_vectorizer_vi.pkl")
        
        # Kiểm tra dữ liệu đã xử lý
        if os.path.exists('final_majors_data_vi.csv'):
            df = pd.read_csv('final_majors_data_vi.csv')
            print(f" Dữ liệu đã xử lý: {len(df)} ngành học (tiếng Việt)")
            print(" Hệ thống đã SẴN SÀNG để sử dụng!")
            return True
    else:
        print("❌ Chưa có mô hình tiếng Việt, tiến hành huấn luyện mới...")
        return False

model_exists = check_existing_model()

## Bước 3: Huấn luyện mô hình mới (nếu cần)

In [ ]:
def train_new_model():
    """Huấn luyện mô hình mới nếu chưa có"""
    print("\n BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH MỚI...")
    
    # Tải dữ liệu từ thư mục data
    try:
        df_coursera = pd.read_csv('data/coursera_course_dataset_v2_no_null.csv')
        df_rankings = pd.read_csv('data/cwurData.csv')
        print("✅ Đã tải dữ liệu thành công từ thư mục data")
    except Exception as e:
        print(f"❌ Lỗi khi tải dữ liệu: {e}")
        return None, None
    
    # Chuẩn bị dữ liệu
    print(" Đang chuẩn bị dữ liệu...")
    
    # Đổi tên cột
    df_coursera = df_coursera.rename(columns={
        'Title': 'major_name',
        'Organization': 'university_name', 
        'Skills': 'major_description'
    })
    
    # Tạo DataFrame mới
    final_courses_df = df_coursera[['major_name', 'university_name', 'major_description']].copy()
    final_courses_df.dropna(subset=['major_description'], inplace=True)
    final_courses_df.drop_duplicates(inplace=True)
    
    # Chuẩn hóa tên
    df_rankings['institution'] = df_rankings['institution'].str.lower()
    final_courses_df['university_name'] = final_courses_df['university_name'].str.lower()
    
    # Hợp nhất dữ liệu
    df_rankings_simple = df_rankings[['institution', 'country', 'score']]
    master_df = pd.merge(
        final_courses_df,
        df_rankings_simple,
        left_on='university_name',
        right_on='institution',
        how='left'
    )
    
    # Xử lý giá trị thiếu
    master_df['score'] = master_df['score'].fillna(master_df['score'].mean())
    master_df['country'] = master_df['country'].fillna('Unknown')
    
    # Tạo DataFrame cuối cùng
    final_df = master_df[['major_name', 'major_description', 'university_name', 'country', 'score']]
    final_df = final_df.rename(columns={'score': 'university_score'})
    final_df = final_df.reset_index(drop=True)
    
    print(f"📈 Đã xử lý xong: {len(final_df)} ngành học")
    
    # Huấn luyện TF-IDF
    print(" Đang huấn luyện mô hình TF-IDF...")
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    major_vectors = vectorizer.fit_transform(final_df['major_description'])
    
    print("✅ Huấn luyện hoàn tất!")
    print(f" Kích thước ma trận: {major_vectors.shape}")
    
    return vectorizer, final_df

# Chỉ huấn luyện nếu chưa có model
if not model_exists:
    vectorizer, final_df = train_new_model()
else:
    # Load model và data hiện có
    vectorizer = joblib.load('tfidf_vectorizer_vi.pkl')
    final_df = pd.read_csv('final_majors_data_vi.csv')
    print("✅ Đã tải mô hình và dữ liệu tiếng Việt hiện có")

## Bước 4: Lưu mô hình (nếu huấn luyện mới)

In [ ]:
def save_trained_model(vectorizer, dataframe):
    """Lưu mô hình đã huấn luyện"""
    print("\n ĐANG LƯU MÔ HÌNH...")
    
    # Lưu vectorizer
    joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
    
    # Lưu dữ liệu đã xử lý
    dataframe.to_csv('final_majors_data.csv', index=False)
    
    # Lưu metadata
    metadata = {
        'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'num_courses': len(dataframe),
        'num_universities': dataframe['university_name'].nunique(),
        'num_features': vectorizer.get_feature_names_out().shape[0]
    }
    joblib.dump(metadata, 'model_metadata.pkl')
    
    print("✅ Đã lưu mô hình thành công!")
    print(f"📁 Các tệp đã lưu:")
    print(f"   - tfidf_vectorizer.pkl (Mô hình TF-IDF)")
    print(f"   - final_majors_data.csv (Dữ liệu đã xử lý)")
    print(f"   - model_metadata.pkl (Thông tin huấn luyện)")

# Lưu model nếu vừa huấn luyện mới
if not model_exists and 'vectorizer' in locals() and 'final_df' in locals():
    save_trained_model(vectorizer, final_df)

## Bước 5: Kiểm tra hệ thống

In [ ]:
def test_recommendation_system():
    """Kiểm tra hệ thống gợi ý"""
    print("\n KIỂM TRA HỆ THỐNG GỢI Ý...")
    
    # Load model và data
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    final_df = pd.read_csv('final_majors_data.csv')
    
    # Vector hóa dữ liệu
    major_vectors = vectorizer.transform(final_df['major_description'])
    
    # Hàm gợi ý
    def get_recommendations(user_interest, min_score=60, top_n=3):
        # Dịch sang tiếng Anh nếu cần
        if any(char in user_interest for char in 'àáâãèéêìíòóôõùúýỳỹỷỵ'):
            try:
                translator = GoogleTranslator(source='vi', target='en')
                user_interest = translator.translate(user_interest)
                print(f"🌐 Đã dịch sang: '{user_interest}'")
            except:
                pass
        
        # Lọc theo điểm
        filtered_df = final_df[final_df['university_score'] >= min_score]
        
        if filtered_df.empty:
            print("❌ Không có ngành học nào phù hợp")
            return None
        
        # Tính độ tương đồng
        user_vector = vectorizer.transform([user_interest])
        filtered_indices = filtered_df.index
        similarities = cosine_similarity(user_vector, major_vectors[filtered_indices])
        
        # Thêm điểm và sắp xếp
        filtered_df = filtered_df.copy()
        filtered_df['similarity'] = similarities[0]
        recommendations = filtered_df.nlargest(top_n, 'similarity')
        
        return recommendations
    
    # Test cases
    test_cases = [
        ("data science", 70, "Khoa học dữ liệu"),
        ("machine learning", 65, "Học máy"),
        ("business management", 60, "Quản trị kinh doanh")
    ]
    
    for interest, score, description in test_cases:
        print(f"\n🔍 Tìm kiếm: {description} (điểm ≥ {score})")
        print("-" * 50)
        
        results = get_recommendations(interest, score, 2)
        
        if results is not None and not results.empty:
            for idx, (_, row) in enumerate(results.iterrows(), 1):
                print(f"{idx}. 🎓 {row['major_name']}")
                print(f"   🏫 {row['university_name'].title()}")
                print(f"   ⭐ Điểm: {row['university_score']:.1f}")
                print(f"   💫 Độ phù hợp: {row['similarity']:.3f}")
                print()
        else:
            print("   Không tìm thấy kết quả")
    
    print("✅ Hệ thống hoạt động tốt!")

# Chạy kiểm tra
test_recommendation_system()

## Bước 6: Tổng kết

In [ ]:
def show_summary():
    """Hiển thị tổng kết hệ thống"""
    print("\n" + "🎯" * 30)
    print("TỔNG KẾT HỆ THỐNG GỢI Ý")
    print("🎯" * 30)
    
    # Load metadata
    if os.path.exists('model_metadata.pkl'):
        metadata = joblib.load('model_metadata.pkl')
        print("📊 THÔNG TIN HUẤN LUYỆN:")
        print(f"   • Thời gian: {metadata['training_date']}")
        print(f"   • Số ngành học: {metadata['num_courses']}")
        print(f"   • Số trường: {metadata['num_universities']}")
        print(f"   • Số từ vựng: {metadata['num_features']}")
    
    # Hiển thị dữ liệu
    df = pd.read_csv('final_majors_data.csv')
    print(f"\n📈 THỐNG KÊ DỮ LIỆU:")
    print(f"   • Tổng số ngành: {len(df)}")
    print(f"   • Số trường duy nhất: {df['university_name'].nunique()}")
    print(f"   • Điểm trường trung bình: {df['university_score'].mean():.1f}")
    
    print(f"\n🚀 HỆ THỐNG ĐÃ SẴN SÀNG!")
    print("💡 Sử dụng hàm get_recommendations() để nhận gợi ý")

# Hiển thị tổng kết
show_summary()

## Bước 7: Hướng dẫn sử dụng

In [ ]:
print("\n📖 HƯỚNG DẪN SỬ DỤNG:")
print("=" * 50)
print("""
Để sử dụng hệ thống gợi ý, copy hàm sau:

def get_course_recommendations(your_interest, min_score=60, top_n=5):
    # Load model
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    final_df = pd.read_csv('final_majors_data.csv')
    major_vectors = vectorizer.transform(final_df['major_description'])
    
    # Dịch nếu là tiếng Việt
    if any(char in your_interest for char in 'àáâãèéêìíòóôõùúýỳỹỷỵ'):
        translator = GoogleTranslator(source='vi', target='en')
        your_interest = translator.translate(your_interest)
    
    # Lọc và tính toán
    filtered_df = final_df[final_df['university_score'] >= min_score]
    user_vector = vectorizer.transform([your_interest])
    filtered_indices = filtered_df.index
    similarities = cosine_similarity(user_vector, major_vectors[filter_indices])
    
    filtered_df = filtered_df.copy()
    filtered_df['similarity'] = similarities[0]
    return filtered_df.nlargest(top_n, 'similarity')

# Ví dụ sử dụng:
results = get_course_recommendations(
    "khoa học dữ liệu và trí tuệ nhân tạo", 
    min_score=70, 
    top_n=3
)
""")

print("\n HUẤN LUYỆN HOÀN TẤT! Hệ thống đã sẵn sàng.")